<a href="https://colab.research.google.com/github/andyarnell/sepal_mgci/blob/master/SDG_15_4_2_Sub_B_Default_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SDG 15.4.2 Subcomponent A: Calculate Global Default Values**

* This script allows batch processing for this indicator for all countries.

* Output is a combined excel file on your Google Drive.

* Runs on the cloud using [Google Colab](https://research.google.com/colaboratory/faq.html)

* Requires: [Google Earth Engine](https://earthengine.google.com/) (GEE) account and project and access to Google Drive


### Install required packages
NB may get warning on ipython version differnces. Then crashing on the next cell. Works ine after this though. So if you rerun that cell all should work after that.

TO DO Check with Daniel if can shift sepal_ui import requirement

In [11]:
# to automatically reload modules.
%load_ext autoreload

# Set to reload all modules before executing code.
%autoreload 2

# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

# List of packages to install if not already installed
packages_to_install = ['ipyvuetify','ee', 'unidecode', 'google-api-python-client',
                      'google-auth-httplib2', 'google-auth-oauthlib','geemap'] #admin_boundaries

# Install necessary packages
for package in packages_to_install:
    install_if_not_exists(package)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
ipyvuetify has been installed.
ee is already installed.
unidecode is already installed.
google-api-python-client has been installed.
google-auth-httplib2 has been installed.
google-auth-oauthlib has been installed.
geemap is already installed.


### Access GitHub repository
Clones repository for SDG 15.4.2 into colab.
Provides functions and lookup tables etc.

In [2]:
# Change the current working directory to "/content".
%cd "/content"

# Clone the GitHub repository "sepal_mgci" into the current directory.
# NB 'fatal' error on reruns are typically just saying it already exists
!git clone https://github.com/sepal-contrib/sepal_mgci

/content
Cloning into 'sepal_mgci'...
remote: Enumerating objects: 2923, done.
remote: Counting objects: 100% (861/861), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 2923 (delta 575), reused 733 (delta 540), pack-reused 2062
Receiving objects: 100% (2923/2923), 4.88 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (1851/1851), done.


In [3]:
# Change current directory to sepal_mgci (i.e. the local copy of the github repository)
%cd "/content/sepal_mgci"

/content/sepal_mgci


##Set parameters


Input parameters

In [5]:
# Google Earth Engine project
gee_project_name = "ee-andyarnellgee" # "insert cloud project here"  # a registered cloud project (if unsure of name see pic here: https://developers.google.com/earth-engine/cloud/assets)


# Admin boundaries asset
admin_asset_id = "FAO/GAUL/2015/level0" # administrative units feature collection

admin_asset_property_name = "ADM0_NAME" # property/column name for selecting admin boundaries (e.g. ISO3 code or country name)


# Land cover assets

# For SUB_B indicator, we need to set the following structure
sub_b_year = {
    "baseline": {
        "base": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000",
            "year": 2000,
        },
        "report": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2015",
            "year": 2015,
        },
    },
    # And the reporting year
    2: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2018", "year": 2018},
}




Output parameters

---



In [6]:
final_report_folder = "sdg_15_4_2_B_combined_report" # folder name in Google Drive for final output (if doesnt exist creates one)

final_report_name = "sdg_15_4_2_B_default_global.xlsx" # file name for final excel output

# export GEE tasks or not
export = False # default: True. Set to False if debugging or limiting accidental re-exporting of tasks

# prints more messages
debug = False # default: False. Set to True if debugging code

Temporary output parameters


In [7]:
stats_csv_folder = "sdg_15_4_2_B_csvs" # for storing stats tables exported from GEE for each admin boundary/AOI

excel_reports_folder = "sdg_15_4_2_B_reports" # for storing formatted excel tables for each admin boundary/AOI

drive_home ="/content/drive/MyDrive/" # Google Drive location. Don't change unless you know this is incorrect

error_log_file_path = drive_home + excel_reports_folder + "/"+"1_error_log" +".csv" # for storing errors


#### Setup Google Earth Engine
Launches access request pop up window

In [8]:
import ee # google earth engine

ee.Authenticate()

ee.Initialize(project=gee_project_name) # NB gee project name is defined in parameters section

### Import packages

In [19]:
from imports import * # import all packages listed in the imports.py script

#### Setup Google Drive
Launches access request pop up window

In [20]:
# for accessing google drive
from google.colab import auth, drive
from googleapiclient.discovery import build

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## SUB INDICATOR B

Create list of boundaries to process

In [21]:
# admin boundary feature collection
admin_boundaries = ee.FeatureCollection(admin_asset_id).limit(10)

#TEMP FILTER FOR TESTING TO LIMIT TO FIRST TEN BOUNDARIES - remove when running properly
admin_boundaries = admin_boundaries.limit(10)

# list to process
list_of_countries = admin_boundaries.aggregate_array(admin_asset_property_name).getInfo()

print ("Length of admin boundaries to process", len(list_of_countries))

list_of_countries = list(set(list_of_countries)) # remove dupicates

print ("Length of distinct admin boundaries to process", (len(set(list_of_countries))))


Length of admin boundaries to process 10
Length of distinct admin boundaries to process 10


Read the default land cover remapping table and convert it to a dictionary

In [22]:
default_map_matrix = map_matrix_to_dict(LC_MAP_MATRIX)


# Set the defauult transition matrix path
default_transition_matrix_path = TRANSITION_MATRIX_FILE

Select years of land cover to process

In [24]:
# extracts the years from the b_years dictionary
years = get_b_years(sub_b_year)

#### Run area statistics within admin boundaries
* Runs for each country and each mountain biobelt
* Gets area of land cover reclassified into the 10 SEAM classes
* Repeat for each year specified


In [26]:
# you can monitor your GEE tasks here : https://code.earthengine.google.com/tasks

create_folder_if_not_exists(stats_csv_folder) # to store outputs in google drive

counter=0 # starting place of counter used to keep track of number of tasks that are being run

for aoi_name in list_of_countries:

    aoi = admin_boundaries.filter(ee.Filter.eq(admin_asset_property_name,aoi_name))#.first()

    # gets areas of landcover in each mountain belt in each country
    # uses reduce_regions function imported from the cloned sepal_mgci git hub repository (see Imports section)
    # pixels counted at native resolution (scale) of input land cover (or DEM if RSA implementation)
    # process = ee.FeatureCollection([
    #     ee.Feature(
    #         None,
    #         reduce_regions(
    #             aoi,
    #             remap_matrix=default_map_matrix,
    #             rsa=False,
    #             # dem=param.DEM_DEFAULT,
    #             dem=DEM_DEFAULT, #default digital elevation model (DEM). Relevant for the real surface area (RSA) implementation.
    #             lc_years= year,
    #             transition_matrix=False
    #         )
    #     ).set("process_id", year[0]["year"])
    #     for year in get_a_years(a_years) # creates GEE images and runs stats on each. Images to run are in the 'a_years" dictionary (above)
    # ])
    process = ee.FeatureCollection([
        ee.Feature(
            None,
            reduce_regions(
                aoi,
                remap_matrix=default_map_matrix,
                rsa=False,
                dem=DEM_DEFAULT, #default digital elevation model (DEM). Relevant for the real surface area (RSA) implementation.
                lc_years= year,
                transition_matrix=default_transition_matrix_path # a matrix of transitions between land cover classes. Used to assess if change between two inputs is to a different (e.g. degraded) state
            )
        ).set("process_id", "_".join([str(y["year"]) for y in year]))
    for year in years # creates GEE images and runs stats on each. Images to run are in the 'b_years" dictionary (above)
    ])
    #make name acceptable for running tasks (i.e., removes special characters)
    task_name = str(sanitize_description(unidecode(aoi_name)))


    task = ee.batch.Export.table.toDrive(
        **{  #asterisks unpack dictionary into keyword arguments format
            "collection": process,
            "description": task_name,
            "fileFormat": "CSV",
            "folder":stats_csv_folder,
            "selectors": [
                "process_id",
                "sub_a",
            ],
        }
    )

    counter+=1

    print (f"\r process {counter}/{len(list_of_countries)} {aoi_name} ", end="") #print in place (remove \r and end="" for verbose version)

    if export:
      task.start()



Folder 'sdg_15_4_2_B_csvs' already exists.
 process 10/10 South Sudan 

# Read, process, and create report tables

#####Manually check your earth engine task status, once the tasks are complete, run the next cell. https://code.earthengine.google.com/tasks

This formats individual excel reports for each country.
See Error_log.csv for missing files/errors

In [ ]:
# Initialize the counter
counter = 0

# to store outputs in google drive
create_folder_if_not_exists(excel_reports_folder)

# Loop over each AOI name in the list of countries
for aoi_name in list_of_countries:
    counter += 1

    # Clean the AOI name
    aoi_name_clean = str(sanitize_description(unidecode(aoi_name)))

    # Construct the file path for the stats CSV file
    stats_csv_file = aoi_name_clean + ".csv"
    stats_csv_file_path = os.path.join(drive_home, stats_csv_folder, stats_csv_file)

    message = f"Process {counter}, {stats_csv_file}"

    try:
        # Read the results from the CSV file and parse it to a dictionary
        dict_results = read_from_csv(stats_csv_file_path)

        details = {
            "geo_area_name": aoi_name,
            "ref_area": " ",
            "source_detail": " ",
        }

        # Generate reports for the sub_a and mtn indicators
        sub_a_reports = [sub_a.get_reports(parse_result(dict_results[year]["sub_a"], single=True), year, **details) for year in single_years]
        mtn_reports = [mntn.get_report(parse_result(dict_results[year]["sub_a"], single=True), year, **details) for year in single_years]

        # Concatenate the mtn reports
        mtn_reports_df = pd.concat(mtn_reports)

        # Concatenate the sub a reports
        er_mtn_grnvi_df = pd.concat([report[0] for report in sub_a_reports])
        er_mtn_grncov_df = pd.concat([report[1] for report in sub_a_reports])

        # Define the output report file path
        report_file_path = os.path.join(drive_home, excel_reports_folder, aoi_name_clean + ".xlsx")

        # Create the Excel file with the reports
        with pd.ExcelWriter(report_file_path) as writer:
            mtn_reports_df.to_excel(writer, sheet_name="Table1_ER_MTN_TOTL", index=False)
            er_mtn_grncov_df.to_excel(writer, sheet_name="Table2_ER_MTN_GRNCOV", index=False)
            er_mtn_grnvi_df.to_excel(writer, sheet_name="Table3_ER_MTN_GRNCVI", index=False)

            # Adjust column widths and alignment for each sheet
            for sheetname in writer.sheets:
                worksheet = writer.sheets[sheetname]
                for col in worksheet.columns:
                    max_length = max(len(str(cell.value)) for cell in col)
                    column = col[0]
                    adjusted_width = max(max_length, len(str(column.value))) + 4
                    worksheet.column_dimensions[get_column_letter(column.column)].width = adjusted_width

                    # Align "obs_value" column to the right
                    if "OBS" in column.value:
                        for cell in col:
                            cell.alignment = Alignment(horizontal="right")

    except Exception as e:
        # If an error occurs, catch the exception and handle it
        message = f"process {counter}, {stats_csv_file}, Error: {e}"

        # Get the current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Write the error message and file name to the error log file
        error_info = pd.DataFrame([[stats_csv_file, str(e), current_time]], columns=['File Name', 'Error Message', 'Time'])

        mode = 'w' if not os.path.exists(error_log_file_path) else 'a'
        header = False if os.path.exists(error_log_file_path) else True

        # Append or write to the error log file
        error_info.to_csv(error_log_file_path, mode=mode, header=header, index=False)

    print(message)


#### Combine excel files

Make a list of files to combine

In [ ]:
# Directory path where Excel reports are stored
directory_path = os.path.join(drive_home, excel_reports_folder)

# List files in the directory with '.xlsx' extension
files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create a list of full file paths
full_file_paths = [os.path.join(directory_path, file) for file in files]

# Print the number of Excel files found in the folder
print(f"Number of Excel files in folder: {len(full_file_paths)}")

# folder to store outputs in google drive
create_folder_if_not_exists(final_report_folder)

# File path for the combined final report
reports_combined_file_path = os.path.join(drive_home, final_report_folder, final_report_name)


##### Run function to combine into a single report

In [ ]:
append_excel_files(file_paths=full_file_paths,num_sheets=3,output_file_path=reports_combined_file_path)

print (f"\n Complete! Output file for SDG 15.4.2 Component A here: {reports_combined_file_path}")